In [1]:
from netgen.csg import CSGeometry, OrthoBrick, Pnt
from ngsolve import *
from ngsolve.internal import *
from xfem import *
from xfem.lsetcurv import *
from math import pi,e
from numpy import linspace

importing ngsxfem-2.1.2504


In [2]:
def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(3)

In [76]:
# physical parameters for linear elastic
mu  = 10
lam = 10

# parameters of DG method
order_u = 1
beta_u = 500

# parameter of ghost penalty
gamma_u = 1

In [77]:
# manufactured solution
u_x = sin(pi*x) * sin(pi*y) * sin(pi*z)
u_y = cos(pi*x) * sin(pi*y) * sin(pi*z)
u_z = sin(pi*x) * cos(pi*y) * sin(pi*z)

exact_u = CF((u_x, u_y, u_z))

# strain tensor
epsilon_xx = u_x.Diff(x)
epsilon_yy = u_y.Diff(y) 
epsilon_zz = u_z.Diff(z) 
epsilon_xy = 0.5*(u_x.Diff(y) +  u_y.Diff(x))
epsilon_yz = 0.5*(u_y.Diff(z) +  u_z.Diff(y))
epsilon_xz = 0.5*(u_x.Diff(z) +  u_z.Diff(x))

# total stress tensor
sigma_xx = lam*(epsilon_xx + epsilon_yy + epsilon_zz) + 2*mu*epsilon_xx 
sigma_yy = lam*(epsilon_xx + epsilon_yy + epsilon_zz) + 2*mu*epsilon_yy 
sigma_zz = lam*(epsilon_xx + epsilon_yy + epsilon_zz) + 2*mu*epsilon_zz
sigma_xy = 2*mu*epsilon_xy
sigma_yz = 2*mu*epsilon_yz
sigma_xz = 2*mu*epsilon_xz


# 右端项 f_x, f_y
f_x = - (sigma_xx.Diff(x) + sigma_xy.Diff(y) + sigma_xz.Diff(z))
f_y = - (sigma_xy.Diff(x) + sigma_yy.Diff(y) + sigma_yz.Diff(z))
f_z = - (sigma_xz.Diff(x) + sigma_yz.Diff(y) + sigma_zz.Diff(z))

fe = CF((f_x, f_y, f_z))

uD = exact_u

In [78]:
# Construct background mesh
# Geometry and Mesh
h0 = 1/13
cube = CSGeometry()
cube.Add(OrthoBrick(Pnt(-1/2,-1/2,-1/2), Pnt(1/2,1/2,1/2)))
mesh = Mesh(cube.GenerateMesh(maxh=h0, quad_dominated=False))
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [79]:
levelset = sqrt(x**2 + y**2 + z**2) - 1/3

# Higher order level set approximation
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=2, threshold=1000,
                                      discontinuous_qn=True)
deformation = lsetmeshadap.CalcDeformation(levelset)
lsetp1 = lsetmeshadap.lset_p1

In [80]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)
# Draw(BitArrayCF(hasneg),mesh)
# Draw(BitArrayCF(hasif),mesh)

# facets used for stabilization:
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
interior_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg)
in_surround_facets = GetElementsWithNeighborFacets(mesh,interior_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets") 
# Draw(BitArrayCF(in_surround_facets), mesh, "surrounding_facets") 


In [81]:
#Unfitted DG spaces
Uhbase = VectorL2(mesh, order=order_u, dirichlet=[], dgjumps=True) # space for displacement
# U = Restrict(Uhbase,ci.GetElementsOfType(HASNEG))
U = Compress(Uhbase, GetDofsOfElements(Uhbase, ci.GetElementsOfType(HASNEG)))
u,v = U.TnT()

In [82]:
# Define the jumps and the averages
n = Normalize(grad(lsetp1)) # outer normal vector on the boundary
# n = 1.0 / Norm(grad(lsetp1)) * grad(lsetp1)
ne = specialcf.normal(mesh.dim) # normal vectors on faces
h = specialcf.mesh_size  

strain_u = Sym(Grad(u))
strain_v = Sym(Grad(v))
mean_stress_u = 0.5*(Stress(Sym(Grad(u)))+Stress(Sym(Grad(u.Other()))))*ne
mean_stress_v = 0.5*(Stress(Sym(Grad(v)))+Stress(Sym(Grad(v.Other()))))*ne
jump_u = u - u.Other()
jump_v = v - v.Other()
 

In [83]:
# integration domains:
domega = dCut(lsetp1, NEG, definedonelements=hasneg, deformation=deformation)
dk = dCut(lsetp1, NEG, skeleton=True, definedonelements=interior_facets,
          deformation=deformation)
ds = dCut(lsetp1, IF, definedonelements=hasif, deformation=deformation)
dw = dFacetPatch(definedonelements=ba_facets, deformation=deformation)

In [84]:
Ah = BilinearForm(U)
# Ae
Ah += 2*mu*InnerProduct(strain_u,strain_v)*domega + lam*div(u)*div(v)*domega \
        - (InnerProduct(mean_stress_u,jump_v) + InnerProduct(mean_stress_v,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dk \
        - (InnerProduct(Stress(Sym(Grad(u)))*n,v) + InnerProduct(Stress(Sym(Grad(v)))*n,u) - beta_u/h*InnerProduct(u,v))*ds
# order=1 i_s 
# Ah += gamma_u * InnerProduct(Grad(u) - Grad(u.Other()),Grad(v) - Grad(v.Other())) * dw
Ah += gamma_u * h * InnerProduct(Grad(u) - Grad(u.Other()),Grad(v) - Grad(v.Other())) * dw

Ah.Assemble()

# r.h.s
lh = LinearForm(U) 
lh += fe*v*domega - InnerProduct(uD,Stress(Sym(Grad(v)))*n)*ds + beta_u/h*uD*v*ds
lh.Assemble()

gfu = GridFunction(U)
gfu.vec.data = Ah.mat.Inverse() * lh.vec

In [85]:
error_u = sqrt(Integrate((gfu - exact_u)**2 * domega, mesh))
print(error_u)

0.0025470451776187894


In [55]:
mask = IfPos(levelset,0,1)

u_cont = GridFunction(U)
u_cont.Set(gfu)     # 将 DG 解作为 CF 插值到 H1
Draw(mask*u_cont, mesh)
Draw(mask*exact_u, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [13]:
kappaminus = CutRatioGF(ci)
kappaminus_values = kappaminus.vec.FV().NumPy()
positive_values = [v for v in kappaminus_values if v > 0]
if positive_values:
    min_value_pythonic = min(positive_values)
    print(f"The smallest cut ratio is: {min_value_pythonic}")
else:
    print("There are no cut elements.")

The smallest cut ratio is: 1.597118794057783e-05


In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds

# 计算最大奇异值
rows,cols,vals = Ah.mat.COO()
A = sp.csr_matrix((vals,(rows,cols)))
condition_number = np.linalg.cond(A.todense())
# _, s_max,_ = svds(A, k=1, which='LM')   # largest magnitude

# # # 计算最小奇异值
# _, s_min,_ = svds(A, k=1, which='SM')   # smallest magnitude

# condition_number = s_max[0] / s_min[0]
print(f"{condition_number:.2e}")
